In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.m2.ssro import ssro, sequence
import scipy.fftpack
import win32com.client #imports the pywin32 library

%matplotlib inline

In [ ]:
def fourier(signal, time):
    """Returns the Fouier transform of a single signal with time."""
    t_step = (time[-1] - time[0])/time.size
    fft = scipy.fftpack.fft(signal) * t_step
    freq = scipy.fftpack.fftfreq(time.size, t_step)
    n = len(fft)
    fft = fft[0:n/2]
    freq = freq[0:n/2]
    return np.abs(fft), freq

frequency = 1.72e9
def read_data(which_measurement='latest'):
    # Get the latest data from the folder.
    if which_measurement == 'latest':
        fol = tb.latest_data('mw_pulse_msmt')#, folder='Z:/data')
    else:
        which_measurement = 20180321120058
        fol = tb.data_from_time(str(which_measurement), folder='D:/measuring/data')#, folder='Z:/data')
    global timestamp
    timestamp = fol[18:33]
    f = sequence.SequenceAnalysis(fol, hdf5_mode='r') 
    waveform = np.array(f.g['oscilloscope_msmt']['data'])
    global number_of_pulses, frequency
    number_of_pulses = int(f.g.attrs['multiplicity'])
    points_per_pulse = int(1e6 // number_of_pulses)
    frequency = f.g.attrs['mw_frq']
    # Make numpy arrays to store the information in and make them global.
    global t, amplitude, fft, freq
    t = np.zeros((number_of_pulses, points_per_pulse))
    amplitude = np.zeros((number_of_pulses, points_per_pulse))
    fft = np.zeros((number_of_pulses, points_per_pulse//2))
    freq = np.zeros((number_of_pulses, points_per_pulse//2))
    # Get the information of all the pulses
    for i in range(number_of_pulses):
        t[i] = waveform[0][i*points_per_pulse:(i+1)*points_per_pulse]
        amplitude[i] = waveform[1][i*points_per_pulse:(i+1)*points_per_pulse]    
        fft[i], freq[i] = fourier(amplitude[i], t[i])

def hermite_pulse(env_amplitude=1, mu=0, T_herm = 1.667e-8, freq_sim=frequency):
    t = np.linspace(-25e-8,25e-8,int(1e4))
    env = env_amplitude*(1-0.667*((t-mu)/T_herm)**2)*np.exp(-((t-mu)/T_herm)**2)
    pulse = env * np.sin(t*2*np.pi*freq_sim)
    fft, freq = fourier(pulse, t)
    return pulse, t, fft, freq
        
def plot_ideal_fft(amplitude=1.25):
    pulse, t, fft, freq = hermite_pulse(amplitude)
    plt.plot(freq, fft, 'r--', label='Ideal')
    
def set_limits():
    ind = np.argmin(np.abs(freq[0] - 1.72e9))
    plt.xlim(1.64e9, 1.8e9)
    plt.ylim(0, 1.4*fft[0, ind])

# Noise of oscilloscope

In [ ]:
read_data()
noise_oscilloscope = np.std(fft)
print "std of fft {:.3e}".format(noise_oscilloscope)
noise_oscilloscope = np.std(amplitude)
print "std of amplitude {:.3e}".format(noise_oscilloscope)

# Analysis of one (first) pulse

In [ ]:
read_data()
t = t[0]
amplitude = amplitude[0]
# Plot waveform
plt.figure(figsize=(8,8))
plt.plot(t, amplitude)
plt.xlabel('s')
plt.ylabel('V')
plt.title('Single Hermite pulse, timestamp : {}'.format(timestamp))
deviation_from_middle = 9e-8
middle = (t[-1]-t[0])/2 + t[0] + deviation_from_middle
plt.xlim(middle-1e-7, middle+1e-7)
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.show()

# Plot the fourier transform
plt.figure(figsize=(8,5))
plt.plot(freq[0], fft[0], label='Data')
plot_ideal_fft()
plt.title('Fourier transform, timestamp : {}'.format(timestamp))
plt.xlabel('Frequency [Hz]')
plt.legend()
set_limits()
# plt.ylim(0,5e-10)
plt.show()

In [ ]:
print t[np.argmax(amplitude)]
print (t[-1]-t[0])/2 + t[0]
print t[np.argmax(amplitude)] - ((t[-1]-t[0])/2 + t[0])

# Analysis of the pulses in one sweep

In [ ]:
read_data()
mean = np.mean(fft[:], axis=0)
std = np.std(fft, axis=0)
plt.figure(figsize=(10,8))
plt.plot(freq[0], mean, 'k', label='Mean')
plt.fill_between(freq[0],  mean - std, mean + std, facecolor='blue', alpha=0.5, label=r'$\pm$ 1 SD')
set_limits()
plot_ideal_fft()
plt.xlabel('f (Hz)')
plt.title('Mean Fourier transform of {} consecutive pulses , timestamp : {}'.format(number_of_pulses, timestamp))
plt.legend()
plt.show()

plt.figure(figsize=(10,3))
plt.plot(freq[0],  std, label="Noise on RF signal")
# plt.plot([1.66e-9, 1.8e-9], [noise_oscilloscope, noise_oscilloscope], 'k--', label="Noise oscilloscope")
plt.xlim(1.64e9, 1.8e9)
plt.ylim(0,5e-10)
plt.xlabel('f (Hz)')
plt.title('Standard deviation of consequtive pulses, timestamp : {}'.format(timestamp))
plt.show()
ind_low = int(np.argmin(np.abs(freq[0] - 1.705e9)))
ind_high = int(np.argmin(np.abs(freq[0] - 1.735e9)))
mean_plateau = np.mean(mean[ind_low:ind_high])
print "The relative mean standard deviation to the peak is {:.3e}.".format(np.mean(std)/mean_plateau)

In [ ]:
read_data()
plt.figure(figsize=(10,8))
for i in range(number_of_pulses):
    plt.plot(freq[i], fft[i], label='{}'.format(i))
set_limits()
# plt.ylim(0,.3e-7)

plt.xlabel('f (Hz)')
plt.title(' Fourier transforms of {} consecutive pulses , timestamp : {}'.format(number_of_pulses, timestamp))
# plt.legend()
plt.show()